In [40]:
import os
import json
import pandas as pd
import traceback

In [41]:
from langchain.chat_models import ChatOpenAI

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
KEY = os.getenv('OPENAI_API_KEY')

In [75]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=0.6)

In [77]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [78]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [79]:
TEMPLATE = TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [80]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [81]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [82]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [83]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [84]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [85]:
from langchain.chains import SequentialChain
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [86]:
file_path = r"C:\Users\amits\mcqgen\data.txt"

In [87]:
file_path

'C:\\Users\\amits\\mcqgen\\data.txt'

In [88]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [89]:
print(TEXT)


“Data science is the study of extracting useful insights from data using scientific methods, statistical techniques, and computational algorithms.“

In today’s world, a large amount of data is generated daily. The main challenge is to deal with this data and extract insights from it to help various organizations and businesses. This is where Data Science comes to the rescue to find patterns and insights.

If you are passionate about data science and want to know all about it, we will answer your questions like what is data science, what tools are used in data science, or what skills are required to become a data scientist. In this article, we will deeply explore data science concepts, why is it important, and how to make a career in data science.

What is Data Science in Simple Words?
Data science is the study to extract useful insights from the data.

Data science involves the process of collecting, analyzing, and interpreting large amounts of data to find patterns, insights, and tre

In [90]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps('RESPONSE_JSON')

'"RESPONSE_JSON"'

In [91]:
NUMBER=5 
SUBJECT="Data Science"
TONE="simple"

In [92]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
“Data science is the study of extracting useful insights from data using scientific methods, statistical techniques, and computational algorithms.“

In today’s world, a large amount of data is generated daily. The main challenge is to deal with this data and extract insights from it to help various organizations and businesses. This is where Data Science comes to the rescue to find patterns and insights.

If you are passionate about data science and want to know all about it, we will answer your questions like what is data science, what tools are used in data science, or what skills are required to become a data scientist. In this article, we will deeply explore data science concepts, why is it important, and how to make a career in data science.

What is Data Science in Simple Words?
Data science is the study to extract useful insights from the data.

Data science involves the 

In [93]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1974
Prompt Tokens:1599
Completion Tokens:375
Total Cost:0.0031485000000000003


In [94]:
response

{'text': '\n“Data science is the study of extracting useful insights from data using scientific methods, statistical techniques, and computational algorithms.“\n\nIn today’s world, a large amount of data is generated daily. The main challenge is to deal with this data and extract insights from it to help various organizations and businesses. This is where Data Science comes to the rescue to find patterns and insights.\n\nIf you are passionate about data science and want to know all about it, we will answer your questions like what is data science, what tools are used in data science, or what skills are required to become a data scientist. In this article, we will deeply explore data science concepts, why is it important, and how to make a career in data science.\n\nWhat is Data Science in Simple Words?\nData science is the study to extract useful insights from the data.\n\nData science involves the process of collecting, analyzing, and interpreting large amounts of data to find pattern

In [95]:
quiz=response.get("quiz")

In [97]:
print(quiz)

{
"1": {
"mcq": "What is the main goal of Data Science?",
"options": {
"a": "To collect data",
"b": "To extract useful insights from data",
"c": "To visualize data",
"d": "To store data"
},
"correct": "b"
},
"2": {
"mcq": "Which programming languages are commonly used in Data Science?",
"options": {
"a": "Java and C++",
"b": "Python and R",
"c": "JavaScript and Ruby",
"d": "PHP and SQL"
},
"correct": "b"
},
"3": {
"mcq": "What is the importance of Machine Learning in Data Science?",
"options": {
"a": "It helps in automating tasks",
"b": "It helps in data visualization",
"c": "It helps in data collection",
"d": "It helps in data storage"
},
"correct": "a"
},
"4": {
"mcq": "Why is knowledge of Statistics important in Data Science?",
"options": {
"a": "To build models",
"b": "To manage databases",
"c": "To understand real-world events with data",
"d": "To automate tasks"
},
"correct": "c"
},
"5": {
"mcq": "What is the purpose of Model Deployment in Data Science?",
"options": {
"a": "To co

In [98]:
quiz=json.loads(quiz)

In [99]:
quiz

{'1': {'mcq': 'What is the main goal of Data Science?',
  'options': {'a': 'To collect data',
   'b': 'To extract useful insights from data',
   'c': 'To visualize data',
   'd': 'To store data'},
  'correct': 'b'},
 '2': {'mcq': 'Which programming languages are commonly used in Data Science?',
  'options': {'a': 'Java and C++',
   'b': 'Python and R',
   'c': 'JavaScript and Ruby',
   'd': 'PHP and SQL'},
  'correct': 'b'},
 '3': {'mcq': 'What is the importance of Machine Learning in Data Science?',
  'options': {'a': 'It helps in automating tasks',
   'b': 'It helps in data visualization',
   'c': 'It helps in data collection',
   'd': 'It helps in data storage'},
  'correct': 'a'},
 '4': {'mcq': 'Why is knowledge of Statistics important in Data Science?',
  'options': {'a': 'To build models',
   'b': 'To manage databases',
   'c': 'To understand real-world events with data',
   'd': 'To automate tasks'},
  'correct': 'c'},
 '5': {'mcq': 'What is the purpose of Model Deployment in Da

In [100]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [101]:
quiz_table_data

[{'MCQ': 'What is the main goal of Data Science?',
  'Choices': 'a: To collect data | b: To extract useful insights from data | c: To visualize data | d: To store data',
  'Correct': 'b'},
 {'MCQ': 'Which programming languages are commonly used in Data Science?',
  'Choices': 'a: Java and C++ | b: Python and R | c: JavaScript and Ruby | d: PHP and SQL',
  'Correct': 'b'},
 {'MCQ': 'What is the importance of Machine Learning in Data Science?',
  'Choices': 'a: It helps in automating tasks | b: It helps in data visualization | c: It helps in data collection | d: It helps in data storage',
  'Correct': 'a'},
 {'MCQ': 'Why is knowledge of Statistics important in Data Science?',
  'Choices': 'a: To build models | b: To manage databases | c: To understand real-world events with data | d: To automate tasks',
  'Correct': 'c'},
 {'MCQ': 'What is the purpose of Model Deployment in Data Science?',
  'Choices': 'a: To collect data | b: To analyze data | c: To make predictions on new datasets | d:

In [102]:
quiz=pd.DataFrame(quiz_table_data)

In [103]:
quiz.to_csv("datascience.csv",index=False)